In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

import pandas as pd
import time

# CONSTANTS
SERVICE = Service(ChromeDriverManager().install())
DRIVER = webdriver.Chrome(service=SERVICE)
URL = "https://store.steampowered.com/specials"
GAMES_CLASS_NAME = "ImpressionTrackedElement"
MAIN_CLASS_NAME = "SaleSectionForCustomCSS"
DF_DICT = {}
# Navigate to the desired URL
DRIVER.get(URL)
time.sleep(5) 

BODY = DRIVER.find_element(By.TAG_NAME, "body")
time.sleep(5)

MAIN_CONTAINER = DRIVER.find_elements(By.CLASS_NAME, MAIN_CLASS_NAME)[2]
time.sleep(5)

In [ ]:
TABS_CONTAINER = None
for tab in TABS_CONTAINER:
    
    while True:
        try:
            DRIVER.execute_script("arguments[0].click();", tab)
            break
        except ElementClickInterceptedException:
            print("Trying")
            tab.send_keys(Keys.PAGE_UP)
            time.sleep(5)  
    time.sleep(5)
    
    DATA_DICT = {
    "Name" : []
    , "Reviews" : []
    , "Reviews Number" : []
    , "Discount" : []
    , "Original Price" : []
    , "Discounted Price" : []
}

    GAMES_CONTAINER = MAIN_CONTAINER.find_elements(By.CLASS_NAME, GAMES_CLASS_NAME)
    time.sleep(5)
    
    for game_container in GAMES_CONTAINER:

        game_container = game_container.text.strip()
        game_container = game_container.split("\n")
    
        DATA_DICT["Name"].append(game_container[0])
        
        reviews = [game_string for game_string in game_container if game_string in REVIEWS]
        reviews = str(reviews).replace('[', '').replace(']', '')
        DATA_DICT["Reviews"].append(reviews)
        
        reviews_number = [game_string for game_string in game_container if "User Reviews" in game_string]
        reviews_number = str(reviews_number).replace('[', '').replace(']', '')
        DATA_DICT["Reviews Number"].append(reviews_number)

        discount = [game_string for game_string in game_container if game_string.endswith("%")]  
        discount = str(discount).replace('[', '').replace(']', '') 
        if not discount:
            discount = ""    
        DATA_DICT["Discount"].append(discount)

        price = [game_string for game_string in game_container if game_string.startswith("$")]

        if price:
            DATA_DICT["Original Price"].append(price[0])
            if len(price)>1:
                DATA_DICT["Discounted Price"].append(price[1]) 
            else:
                DATA_DICT["Discounted Price"].append("")           
        else:
            DATA_DICT["Original Price"].append("Coming Soon")
            DATA_DICT["Discounted Price"].append("")

    DF_DICT[tab.text] = pd.DataFrame.from_dict(DATA_DICT)